<a href="https://colab.research.google.com/github/KV721/UrbanGAN/blob/main/RAG_Ex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install huggingface_hub transformers sentence-transformers
!pip install docling
!pip install chromadb
!pip install google-genai

In [ ]:
import os
from pathlib import Path

from docling.document_converter import DocumentConverter
from docling.chunking import HybridChunker
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.utils import embedding_functions
from google import genai

In [ ]:
os.environ['GEMINI_API_KEY'] =""

In [ ]:
converter = DocumentConverter()
document_path = "/content/set_transfomer.pdf"

result = converter.convert(document_path)
full_text = result.document.export_to_markdown()

In [ ]:
# 🟢 TODO 2: Complete the simple chunking function.
# Split `text` into overlapping chunks of size `chunk_size`
# with `overlap` characters shared between consecutive chunks.
def chunk_text(text, chunk_size, overlap):
    """Split text into chunks with overlap."""
    chunks = []
    start = 0
    text_length = len(text)

    while start < text_length:
        end = start + chunk_size
        # TODO: Append the chunk from `start` to `end`

        # TODO: Move `start` forward by (chunk_size - overlap)

    return chunks

simple_chunks = chunk_text(full_text, chunk_size=256, overlap=50)
print(f"✓ Simple chunks created: {len(simple_chunks)}\n")

for i, chunk in enumerate(simple_chunks[:3]):
    print(f"\n--- Simple Chunk {i+1} ---")
    print(chunk[:200] + "..." if len(chunk) > 200 else chunk)

In [ ]:
# 🟡 TODO 3: Initialize the HybridChunker with the correct parameters.
# Use:
#   tokenizer="sentence-transformers/all-MiniLM-L6-v2"
#   max_tokens=256
#   merge_peers=False
chunker = HybridChunker(
    # TODO: fill in the three parameters
)

chunk_iter = chunker.chunk(dl_doc=result.document)
hybrid_chunks = list(chunk_iter)

print(f"✓ Hybrid chunks created: {len(hybrid_chunks)}\n")

for i, chunk in enumerate(hybrid_chunks[:3]):
    print(f"\n--- Hybrid Chunk {i+1} ---")
    chunk_text_str = chunk.text if hasattr(chunk, 'text') else str(chunk)
    print(f"Content: {chunk_text_str[:200]}..." if len(chunk_text_str) > 200 else chunk_text_str)

In [ ]:
hybrid_chunks_text = [
    chunk.text if hasattr(chunk, 'text') else str(chunk)
    for chunk in hybrid_chunks
]

print(f"✓ Total simple chunks (for comparison): {len(simple_chunks)}")
print(f"✓ Total hybrid chunks: {len(hybrid_chunks_text)}")

In [ ]:
EMBEDDING_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"

# 🟡 TODO 4: Load the SentenceTransformer model using EMBEDDING_MODEL_NAME
# Hint: use SentenceTransformer(...)
embedding_model = # TODO

# Test it
test_text = "This is a test sentence for embeddings."
test_embedding = embedding_model.encode(test_text)

print(f"✓ Embedding dimension: {len(test_embedding)}")
print(f"✓ Sample embedding (first 2 values): {test_embedding[:2]}")

In [ ]:
chroma_client = chromadb.PersistentClient(path="/content/chroma_db")

sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name=EMBEDDING_MODEL_NAME
)

collection_name = "rag_documents"

try:
    chroma_client.delete_collection(name=collection_name)
except:
    pass

# 🟡 TODO 5: Create a new ChromaDB collection.
# Use chroma_client.create_collection(...) with parameters:
#   - name=collection_name
#   - embedding_function=sentence_transformer_ef
collection = # TODO

In [ ]:
# 🟡 TODO 6: Prepare IDs and metadata, then add documents to the collection.
# Each chunk needs a unique ID like "chunk_0", "chunk_1", etc.
# Metadata should include chunk_index, source, and chunking_method.

ids = [f"chunk_{i}" for i in range(len(hybrid_chunks_text))]
metadatas = [{"chunk_index": i, "source": document_path, "chunking_method": "hybrid"} for i in range(len(hybrid_chunks_text))]

print("Adding documents to ChromaDB...")

# TODO: Use collection.add(...) with documents, ids, and metadatas
collection.add(
    # TODO: fill in the two arguments
    documents=hybrid_chunks_text
)

print(f"\n✓ Successfully stored {len(hybrid_chunks_text)} chunks in ChromaDB")
print(f"✓ Collection now contains {collection.count()} documents")

In [ ]:
# 🟡 TODO 7: Implement the search function using ChromaDB's query method.
def search_documents(query, n_results=3):
    """Search the vector store for chunks most relevant to the query."""
    # TODO: Use collection.query(...) to search
    # Hint: pass query_texts=[query] and n_results=n_results
    results = # TODO

    return results

In [ ]:
client = genai.Client()

test_response = client.models.generate_content(
    model="gemini-3-flash-preview",
    contents="Explain how AI works in a few words"
)
print("Gemini Flash Model Test:")
print(test_response.text)
print("\n✓ Gemini client initialized successfully!")

In [ ]:
# 🔴 TODO 8: Build the full RAG pipeline function.
# This is the capstone — tie retrieval, augmentation, and generation together.
def rag_query(question, n_results=3):
    """
    Complete RAG pipeline:
    1. Retrieve relevant documents
    2. Augment the prompt with retrieved context
    3. Generate answer using Gemini
    """
    print(f"Question: {question}\n")

    # Step 1: Retrieve relevant documents
    # TODO: Call search_documents function with the question
    search_results = # TODO

    retrieved_docs = search_results['documents'][0]
    print(f"✓ Retrieved {len(retrieved_docs)} relevant chunks\n")

    # Step 2: Build context string from retrieved documents
    # TODO: Join the retrieved docs into a single context string.
    # Format each as "[Context i]\n<doc text>" separated by double newlines.
    context = # TODO

    # Step 3: Create the augmented prompt
    # 🔴 TODO 9 (Prompt Engineering): Write a prompt that instructs the model to:
    #   - Answer based ONLY on the provided context
    #   - Admit when the context is insufficient
    #   - Be clear and concise
    prompt = f"""
    # TODO: Write your prompt here. Include {context} and {question}.
    """

    # Step 4: Generate response
    # TODO: Use client.models.generate_content(...) with model="gemini-3-flash-preview"
    response = # TODO

    print("✓ Answer generated!\n")

    return {
        "question": question,
        "answer": response.text,
        "retrieved_chunks": retrieved_docs,
        "distances": search_results['distances'][0]
    }

In [ ]:
example_questions = [
    "What is the main topic discussed in the document?",
    "Can you summarize the key points?",
    "What are the important details mentioned?"
]

print("=" * 80)
print("RAG SYSTEM DEMO")
print("=" * 80)

for i, question in enumerate(example_questions, 1):
    print(f"\n{'='*80}")
    print(f"EXAMPLE {i}")
    print(f"{'='*80}\n")

    result = rag_query(question, n_results=3)

    print("ANSWER:")
    print("-" * 80)
    print(result['answer'])
    print("-" * 80)

    print(f"\nRETRIEVED CONTEXT CHUNKS:")
    for j, (chunk, distance) in enumerate(
        zip(result['retrieved_chunks'], result['distances']), 1
    ):
        print(f"\nChunk {j} (Similarity Distance: {distance:.4f}):")
        print(f"{chunk[:200]}...")